In [8]:
# Question 1
import random

rows = 10
cols = 10
parking_lot  = []
charging_stations = [(1,1),(2,8),(6,6),(7,3),(9,9)]
entrance = (5,5)
fitness_score = dict()
whole_population = []
population_size = 50
fitness_of_whole_population = []


def make_parking_lot(rows, cols):
  parking_lot = []
  for i in range(rows):
    row = []
    for j in range(cols):
      row.append(0)
    parking_lot.append(row)

  for x,y in charging_stations:
      parking_lot[x][y] = 83
  parking_lot[entrance[0]][entrance[1]] = 65

  return parking_lot


#______________________________________________________#
def print_parking_lot(parking_lot, column_width=4):

    print("\n".join([" ".join([f"{item:>{column_width}}" for item in row]) for row in parking_lot]))


def randomize_pop (parking_lot):
  cars_parked = 1
  while (cars_parked<=50):
    x = random.randint(0,9)
    y = random.randint(0,9)
    if parking_lot[x][y] == 0:
      parking_lot[x][y] = cars_parked
      cars_parked += 1
  return parking_lot

def initiate_population(population_size):
  for i in range(population_size):
    parking_lot = make_parking_lot(rows, cols)
    parking_lot = randomize_pop(parking_lot)
    whole_population.append(parking_lot)

def print_whole_population(population_size):
  for i in range(population_size):
    print(f"Parking Lot {i+1}:")
    print_parking_lot(whole_population[i])
    print("\n")
    print(fitness_score[i+1])




def eculadian_distance_entrance(x1, y1):
  return ((x1-5)**2 + (y1-5)**2)**0.5

def eculadian_distance_charging_station(x1, y1, x2, y2):
  return ((x1-x2)**2 + (y1-y2)**2)**0.5

def sort_fitness_score(fitness_score) :

  fitness_score= dict(sorted(fitness_score.items(), key=lambda item: item[0]))
  return fitness_score


def fitness(parking_lot):
  fitness_score = dict()


  for i in range(rows):
        for j in range(cols):
            if parking_lot[i][j] != 0 and  parking_lot[i][j] != 83 and parking_lot[i][j] != 65:

                current_score=0
                car = parking_lot[i][j]
                if car <=10:
                    distance_list = []
                    for x,y in charging_stations:
                      distance = eculadian_distance_charging_station(i, j, x, y)
                      distance_list.append(distance)
                    current_score += min(distance_list)
                    if min(distance_list) > 4:
                      current_score += 10
                    current_score += eculadian_distance_entrance(i, j)
                    fitness_score[car] = current_score

                elif car > 10 and car <=15:
                    current_score += eculadian_distance_entrance(i, j)
                    if current_score > 3:
                      current_score += 10
                    fitness_score[car] = current_score

                else:
                    current_score += eculadian_distance_entrance(i, j)
                    fitness_score[car] = current_score
  fitness_score = sort_fitness_score(fitness_score)
  return fitness_score




def calculate_pop_fitness(population_size):
  global fitness_of_whole_population
  global fitness_score
  count = 1
  for parking_lot in whole_population:
    fitness_of_whole_population.append(fitness(parking_lot))

  for i in fitness_of_whole_population:
     sum =0
     for j in i.values():
        sum += j
     fitness_score[count] = sum
     count += 1




initiate_population(population_size)
calculate_pop_fitness(population_size)


In [9]:
#question 1 (continued)
import random
def selection():
  global whole_population
  global fitness_of_whole_population
  global fitness_score

  dsc_sorted =dict(sorted(fitness_score.items(), key=lambda item: item[1]))
  mother_index= list(dsc_sorted.keys())[0] -1
  father_index= list(dsc_sorted.keys())[1] -1
  mother = whole_population[mother_index]
  father = whole_population[father_index]
  return mother, father


def crossover_and_add(mother, father):
  global whole_population
  global fitness_of_whole_population
  global fitness_score
  global population_size
  global rows ,cols
  mask_value = 0.50
  child= []
  for i in range(rows):
    row = []
    for j in range(cols):
      prob = random.random()
      if prob < mask_value:
        row.append(mother[i][j])
      else:
        row.append(father[i][j])
    child.append(row)

  whole_population.append(child)
  fitness_of_whole_population.append(fitness(child))
  fitness_score[len(whole_population)] = sum(fitness(child).values())
  population_size += 1


def mutation():
  global whole_population
  global fitness_of_whole_population
  global fitness_score

  mask_value = 0.5

  random_number = random.randint(0, population_size-1)
  random_parking_lot = whole_population[random_number]

  for total_mutations in range(4):
    x = random.randint(0,9)
    y = random.randint(0,9)

    x2 = random.randint(0,9)
    y2 = random.randint(0,9)
    if (random_parking_lot[x][y] != 0 and random_parking_lot[x][y] != 83 and random_parking_lot[x][y] != 65) and (random_parking_lot[x2][y2] != 0 and random_parking_lot[x2][y2] != 83 and random_parking_lot[x2][y2] != 65):
                prob = random.random()
                if prob < mask_value:
                    random_parking_lot[x][y], random_parking_lot[x2][y2] = random_parking_lot[x2][y2], random_parking_lot[x][y] #swap


    fitness_of_whole_population[random_number] = fitness(random_parking_lot)
    fitness_score[random_number+1] = sum(fitness(random_parking_lot).values())
    whole_population[random_number] = random_parking_lot

def top_parking_lot():
  global whole_population
  global fitness_of_whole_population
  global fitness_score
  dsc_sorted =dict(sorted(fitness_score.items(), key=lambda item: item[1]))

  return list(dsc_sorted.keys())[0] -1
def evaluation ():
  global whole_population
  global fitness_of_whole_population
  global fitness_score
  global population_size
  best_one_index = top_parking_lot()
  criteria  =0
  while (criteria < 20):
    mother, father = selection()
    crossover_and_add(mother, father)
    mutation()
    potential_best_one_index = top_parking_lot()
    if best_one_index == potential_best_one_index:
      criteria += 1
    else:
      best_one_index = potential_best_one_index
      criteria = 0

  best_one_index = top_parking_lot()
  print(f"Best Parking Lot: {best_one_index+1}")
  print(f"Fitness score:{fitness_score[best_one_index+1]}")
  print_parking_lot(whole_population[best_one_index])
  print("\n")



evaluation()
print("_____________________________________________________")
print("other candiates fitness score")
print(fitness_score)


Best Parking Lot: 57
Fitness score:149.06639196873655
  46    0   50    0    0   44    2   26    0    0
  24   83    0   34   45    0    9   42    0    0
   0    0    0    0   38    0    0   28   83   32
   0    0    0    0    0   31   12   42    0    0
   0   33    0   25    0    0    0   14   18    0
   0   22    0    0    1   65   26    5   47   18
   0   40   34   48   35    0   83   10   28    0
   6   24    0   83   39    0   22   46   21    0
   0    0    0    0    0    0   43   30   29   13
   0   17    0   48   29    0   33    3    0   83


_____________________________________________________
other candiates fitness score
{1: 235.63833588669982, 2: 278.3631266920628, 3: 263.2298487282172, 4: 258.92387430346463, 5: 255.16308914748595, 6: 260.0275237846678, 7: 227.90388281023903, 8: 222.7975870912582, 9: 251.49831827313307, 10: 258.20688875541316, 11: 230.11366714990564, 12: 262.6088538745069, 13: 249.71209658922604, 14: 233.07513653785156, 15: 254.85053568674218, 16: 261.01037

In [6]:
#question 2

import random

strategies = {
    1: {"expected_return": 0.5, "risk_level": 1, "max_drawdown": -1},
    2: {"expected_return": 1.2, "risk_level": 2, "max_drawdown": -2.5},
    3: {"expected_return": 2.5, "risk_level": 3, "max_drawdown": -5},
    4: {"expected_return": 5.0, "risk_level": 4, "max_drawdown": -10},
    5: {"expected_return": 10.0, "risk_level": 5, "max_drawdown": -15}
}

chromosomes_house = []
fitness_score_house = []
population_size= 100
#Chromosome Representation
def make_population(population_size):
  global chromosomes_house
  for i in range(population_size):
      count = 0
      chromosome= []
      while count <30 :
        value =random.randint(1,5)
        if len(chromosome) > 0 :
          if value != chromosome[-1]:
            chromosome.append(value)
            count += 1
        else:
          chromosome.append(value)
          count += 1
      chromosomes_house.append(chromosome)

def fitness_function(chromosome):
        expected_return = 0
        risk_level = 0
        max_drawdown = 0
        single_dropdown = 0
        ri = 0
        penalties = 0
        for strategy in chromosome:

          expected_return = strategies[strategy]["expected_return"]

          single_drawdown = strategies[strategy]["max_drawdown"]

          ri += expected_return
          max_drawdown += single_drawdown

          if single_drawdown < -0.15 :
            penalties += 0.10

          if strategy > 0 and chromosome[strategy] == chromosome[strategy - 1]:
                  penalties += 0.05

          if max_drawdown < -0.30 :
            penalties += 0.20
        fitness_score = ri - penalties
        return fitness_score


def fitness_for_population(population_size):
  global chromosomes_house
  global fitness_score_house
  for chromosome in chromosomes_house:
    fitness_score_house.append(fitness_function(chromosome))



def selection():
  global chromosomes_house
  global fitness_score_house
  mother = []
  father= []
  random_list = random.sample(range(0, population_size), 5)
  random_chromosomes = []
  random_fitness_score = []

  for index in random_list:
    random_chromosomes.append(chromosomes_house[index])
  for index in random_list:
    random_fitness_score.append(fitness_score_house[index])

  mother = random_chromosomes[random_fitness_score.index(max(random_fitness_score))]
  random_fitness_score.remove(max(random_fitness_score))

  father  = random_chromosomes[random_fitness_score.index(max(random_fitness_score))]

  return mother ,father


def crossover(mother, father):
  point = random.randint(1, len(mother)-1)

  offspring1= mother[:point] + father[point:]
  offspring2 = father[:point] + mother[point:]

  fit1 = fitness_function(offspring1)
  fit2 = fitness_function(offspring2)

  if fit1 > fit2:
    return offspring1
  else:
    return offspring2



def mutation(offspring):
  mask = 0.05
  for i in range (len(offspring)) :
    prob = random.random()
    if prob <mask :
      new_val = random.randint(1,5)
      if i > 0 and i < len(offspring)-1:
        if new_val != offspring[i-1] and new_val != offspring[i+1]:
          offspring[i] = new_val
      elif i == 0:
        if new_val != offspring[i+1]:
          offspring[i] = new_val
      elif i == len(offspring)-1:
        if new_val != offspring[i-1]:
          offspring[i] = new_val
    return offspring

def print_population():
  global chromosomes_house
  global fitness_score_house
  for i in range(population_size):
    print(f"Chromosome {i+1}: {chromosomes_house[i]} Fitness Score: {fitness_score_house[i]}")
    print("_________________________________________________________________________________")


def replace (offspring):
  global chromosomes_house
  global fitness_score_house
  worst_index = fitness_score_house.index(min(fitness_score_house))
  chromosomes_house[worst_index] = offspring
  fitness_score_house[worst_index] = fitness_function(offspring)



def profit_and_risk(chromosome):
  expected_return = 0
  risk_level = 0
  total_drowdown = 0
  for strategy in chromosome:
    expected_return += strategies[strategy]["expected_return"]
    total_drowdown += strategies[strategy]["max_drawdown"]
  return expected_return,total_drowdown






def evaluation():
  global chromosomes_house
  global fitness_score_house
  for i in range(100):
    mother,father = selection()
    offspring = crossover(mother, father)
    offspring = mutation(offspring)
    replace(offspring)
  best_index = fitness_score_house.index(max(fitness_score_house))
  print(f"Best Chromosome: {chromosomes_house[best_index]} Fitness Score: {fitness_score_house[best_index]}")
  expected_return,total_drowdown = profit_and_risk(chromosomes_house[best_index])
  print(f"Expected Return: {expected_return}%")
  print(f"Total Drowdown: {total_drowdown}%")



make_population(population_size)
fitness_for_population(population_size)
evaluation()
print("ALL canidates")
print_population()

Best Chromosome: [5, 3, 5, 2, 5, 3, 4, 5, 3, 2, 5, 2, 5, 3, 5, 4, 3, 5, 4, 5, 4, 5, 4, 5, 2, 1, 2, 5, 4, 4] Fitness Score: 165.0
Expected Return: 174.0%
Total Drowdown: -288.5%
ALL canidates
Chromosome 1: [4, 1, 4, 2, 3, 5, 4, 5, 1, 5, 1, 3, 1, 2, 1, 5, 3, 5, 4, 5, 4, 5, 4, 5, 2, 1, 2, 5, 4, 1] Fitness Score: 131.8
_________________________________________________________________________________
Chromosome 2: [5, 3, 4, 3, 5, 1, 4, 5, 1, 5, 1, 3, 1, 2, 1, 5, 3, 5, 4, 5, 4, 5, 4, 5, 2, 1, 2, 5, 4, 1] Fitness Score: 138.09999999999997
_________________________________________________________________________________
Chromosome 3: [5, 4, 5, 3, 2, 4, 1, 3, 5, 3, 1, 3, 4, 5, 3, 5, 2, 1, 2, 5, 3, 5, 4, 1, 5, 2, 4, 5, 1, 5] Fitness Score: 138.3
_________________________________________________________________________________
Chromosome 4: [5, 4, 1, 5, 3, 2, 5, 2, 4, 5, 2, 5, 1, 5, 1, 3, 1, 5, 4, 3, 2, 1, 5, 4, 5, 2, 4, 5, 1, 5] Fitness Score: 142.5
______________________________________________

In [5]:

import random
#question 3

rows = 3
sharjeel = (2,0)
exit = (0,49)
total_mafia_blocks = 45
cols = 50
grid = []
total_instances =10
sequences = []
fitness_scores = []

MOVES = {
    "F": (0, 1),      # FORWARD
    "B": (0, -1),     # BACKWARD
    "J": (-1, 0),     # JUMP
    "SD": (1, 0),     # STEP_DOWN
    "SJ": (-2, 0),    # SUPER_JUMP
    "D": (0, 2)       # DASH
}


def make_stronghold(rows, cols):
    count = 0
    for i in range(rows):
      row = []
      for j in range(cols):
          row.append('-')
      grid.append(row)

    while count < total_mafia_blocks:
      x = random.randint(0,rows-1)
      y = random.randint(0,cols-1)
      if grid[x][y] == '-':
        count += 1
        grid[x][y] = 'M'

    grid[sharjeel[0]][sharjeel[1]] = 'S'
    grid[exit[0]][exit[1]] = 'E'


def manhatten_distance(x1, y1):
    return abs(x1 -0) + abs(y1 - 49)

def fitness(Sequence):
    score = 0
    player = (2,0)
    prev_distance = manhatten_distance(player[0], player[1])
    for move in Sequence:
        player = (player[0] + MOVES[move][0], player[1] + MOVES[move][1])
        current_distance = manhatten_distance(player[0], player[1])
        if current_distance < prev_distance:
            score += 10
        else :
          score -= 5
        prev_distance = current_distance
    return score

def print_stronghold(grid):
    for row in grid:
        print(' '.join(row))
    print("----------------------------------------------")


def generate_sequence():
    global exit
    Sequence = []
    total_steps = 70
    count = 0

    player = (2,0)

    while count < total_steps and player != exit :
        move = random.choice(list(MOVES.keys()))
        potential_pos = (player[0] + MOVES[move][0], player[1] + MOVES[move][1])
        if potential_pos[0] < 0 or potential_pos[0] >= rows or potential_pos[1] < 0 or potential_pos[1] >= cols or grid[potential_pos[0]][potential_pos[1]] == 'M':
            continue
        Sequence.append(move)
        player = (player[0] + MOVES[move][0], player[1] + MOVES[move][1])
        count += 1
    return Sequence


def make_instances_and_fitness(total_instances):
    global sequences
    for i in range(total_instances):
        sequences.append(generate_sequence())

    for i in sequences:
        value = fitness(i)
        fitness_scores.append(value)

def selection():
  global sequences
  global fitness_scores
  mother = []
  father= []
  temp  = fitness_scores.copy()
  best_index_1 = temp.index(max(temp))
  mother = sequences[best_index_1]
  temp.remove(max(temp))
  best_index_2 = temp.index(max(temp))
  if (best_index_1 == best_index_2):
      temp.remove(max(temp))
  best_index_2 = temp.index(max(temp))
  father = sequences[best_index_2]

  return mother ,father


def crossover(mother, father):
    mask_value = 0.5
    offspring = []
    player = (2, 0)
    min_steps = min(len(mother), len(father))

    for i in range(min_steps):
        prob = random.random()


        potential_pos_mother = (player[0] + MOVES[mother[i]][0], player[1] + MOVES[mother[i]][1])
        potential_pos_father = (player[0] + MOVES[father[i]][0], player[1] + MOVES[father[i]][1])

        if prob < mask_value:

            if 0 <= potential_pos_mother[0] < rows and 0 <= potential_pos_mother[1] < cols and grid[potential_pos_mother[0]][potential_pos_mother[1]] != 'M':
                offspring.append(mother[i])
                player = potential_pos_mother

            elif 0 <= potential_pos_father[0] < rows and 0 <= potential_pos_father[1] < cols and grid[potential_pos_father[0]][potential_pos_father[1]] != 'M':
                offspring.append(father[i])
                player = potential_pos_father

            else:
                continue
        else:

            if 0 <= potential_pos_father[0] < rows and 0 <= potential_pos_father[1] < cols and grid[potential_pos_father[0]][potential_pos_father[1]] != 'M':
                offspring.append(father[i])
                player = potential_pos_father
            elif 0 <= potential_pos_mother[0] < rows and 0 <= potential_pos_mother[1] < cols and grid[potential_pos_mother[0]][potential_pos_mother[1]] != 'M':
                offspring.append(mother[i])
                player = potential_pos_mother

            else:
                continue
    fit = fitness(offspring)
    worst_fitness = min(fitness_scores)
    if fit > worst_fitness:
        sequences[fitness_scores.index(worst_fitness)] = offspring
        fitness_scores[fitness_scores.index(worst_fitness)] = fit


def mutation():
    global sequences
    global fitness_scores
    player = (2, 0)
    mask_value = 0.25
    random_indexes = random.sample(range(0, total_instances), 3)

    for i in random_indexes:
        good_genes = True
        for j in range(len(sequences[i])):
            prob = random.random()
            move_type = sequences[i][j]
            player = (player[0] + MOVES[move_type][0], player[1] + MOVES[move_type][1])


            if prob < mask_value:
                    move = random.choice(list(MOVES.keys()))
                    potential_pos = (player[0] + MOVES[move][0], player[1] + MOVES[move][1])
                    if potential_pos[0] < 0 or potential_pos[0] >= rows or potential_pos[1] < 0 or potential_pos[1] >= cols or grid[potential_pos[0]][potential_pos[1]] == 'M':
                        continue
                    else:

                        sequences[i][j] = move
                        player = potential_pos
                        good_genes = False
                        break


        fitness_scores[i] = fitness(sequences[i])

def run_thorugh_all_moves():
    global sequences
    global exit
    for sequence in sequences:
        player = (2, 0)
        for move in sequence:
            player = (player[0] + MOVES[move][0], player[1] + MOVES[move][1])
        if player == exit:
            return True
    return False


def evaluation():
    count = 0
    while not run_thorugh_all_moves() and count <600:
        mother,father = selection()
        crossover(mother, father)
        mutation()
        count += 1
    if count == 600:
        print("Sharjeel didnt escape")
    else :
      print("Sharjeel escaped!!")

    best_index = fitness_scores.index(max(fitness_scores))
    print("Best sequence:", sequences[best_index])
    print("Best fitness score:", fitness_scores[best_index])
    print("------------------------------------------------------")
    print("Other candidates")
    for i in range(len(sequences)):
        if i != best_index:
            print("Sequence:", sequences[i])
            print("Fitness score:", fitness_scores[i])





make_stronghold(rows, cols)
print_stronghold(grid)
make_instances_and_fitness(total_instances)

evaluation()






- - M M M M - M - - - - - - - M - - - - M - - M - M - - - - M - M - M M - - - - M - M M - M - - - E
M M M - - - M - - M - - - - - - - - M - - - - - - - - M - - - - M - M - - M - M - - - - - M - M - M
S - - - M - M - - M - - - M M - - - M - - - - - M - - - - - - - - - M M M M - - - - - M - - - M - M
----------------------------------------------
Sharjeel didnt escape
Best sequence: ['F', 'D', 'D', 'J', 'J', 'F', 'SD', 'J', 'D', 'F', 'D', 'D', 'F', 'J', 'F', 'F', 'F', 'D', 'F', 'SD', 'SD', 'F', 'D', 'F', 'B', 'F', 'J', 'SD', 'D', 'F', 'F', 'F', 'F', 'F', 'D', 'F']
Best fitness score: 285
------------------------------------------------------
Other candidates
Sequence: ['F', 'SD', 'B', 'B', 'B', 'SJ', 'SJ', 'SD', 'F', 'F', 'B', 'J', 'SJ', 'D', 'F', 'B', 'D', 'SD', 'F', 'J', 'F', 'SD', 'F', 'SJ', 'F', 'F', 'SJ', 'F', 'B', 'F', 'B', 'F', 'SD', 'J', 'SD', 'D', 'F', 'F', 'B', 'F', 'F', 'J', 'SD', 'SD', 'D', 'D', 'D', 'F', 'D', 'B', 'J', 'F', 'SD', 'D']
Fitness score: 255
Sequence: ['SD', 'D',

In [3]:
# question 4
# the input data
import random
time_matrix =  []
cost_factor = 0.25
popularity_factor = 0.25
travel_day_fitness = []
travel_chromosomes = []
visited_attractions = {
    "Badshahi Mosque": [5, 10, 2, 0], #cost,popularity,visit,umm visits_count
    "Lahore Fort": [8, 9, 2, 0],
    "Faisal Mosque": [5, 9, 2, 0],
    "Karimabad (Hunza Valley)": [1, 10, 3, 0],
    "Karimabad Baltit Fort": [5, 8, 2, 0],
    "Shah Faisal Mosque": [1, 9, 1, 0],
    "Minar-e-Pakistan": [2, 9, 1, 0],
    "Wazir Khan Mosque": [3, 8, 2, 0],
    "Kaghan Valley": [1, 9, 4, 0],
    "Murree Hills": [1, 8, 2, 0],
    "Ziarat Juniper Forest": [1, 7, 3, 0],
    "Mohenjo-Daro": [10, 9, 3, 0],
    "Lahore Museum": [5, 8, 2, 0],
    "Ranikot Fort": [1, 7, 2, 0],
    "Makli Necropolis": [5, 8, 2, 0],
    "Skardu Valley": [1, 10, 4, 0],
    "Fairy Meadows": [1, 9, 3, 0],
    "Karakoram Highway": [1, 10, 5, 0],
    "Lake Saif ul Malook": [1, 9, 3, 0],
    "Shandur Top": [1, 8, 3, 0],
    "Beshigram Valley": [1, 8, 3, 0],
    "Naltar Valley": [1, 8, 3, 0],
    "Swat Valley": [1, 9, 4, 0],
    "Chitral Fort": [5, 8, 2, 0],
    "Gorakh Hill Station": [1, 8, 3, 0],
    "Pir Sohawa": [1, 7, 2, 0],
    "Sundar Das Temple": [2, 7, 2, 0],
    "Buddhist Monastery": [8, 7, 2, 0],
    "Shalimar Gardens": [8, 8, 2, 0],
    "M2 Motorway Viewpoints": [1, 8, 1, 0],
    "Tomb of Jahangir": [5, 8, 1, 0],
    "Tomb of Allama Iqbal": [5, 9, 2, 0],
    "Chaukhandi Tombs": [5, 7, 2, 0],
    "Kirthar National Park": [1, 7, 3, 0],
    "Lulusar Lake": [1, 9, 3, 0],
    "Kund Malir Beach": [1, 8, 3, 0],
    "Manthoka Waterfall": [1, 8, 3, 0],
    "Siri Paye": [1, 9, 2, 0],
    "Mazar-e-Quaid": [5, 9, 2, 0],
    "Pakistani National Museum of Ethnology": [3, 7, 2, 0],
    "Chilam Chowki": [1, 7, 2, 0],
    "Faisalabad Clock Tower": [5, 7, 2, 0],
    "K2 Base Camp": [30, 10, 7, 0],
    "Banu Temple": [2, 7, 2, 0],
    "Rohri Hills": [1, 6, 3, 0],
    "Makran Coastal Highway": [1, 10, 2, 0],
    "Hingol National Park": [1, 9, 3, 0],
    "Harappa Archaeological Site": [10, 8, 3, 0],
    "Gwadar Port": [1, 7, 3, 0],
    "Hanna lake Quetta": [4, 3, 2, 0],
}



def get_key_index(key):
  keys = list(visited_attractions.keys())
  return keys.index(key)


def create_matrix():
  global time_matrix
  for i in range (50):
    row = []
    for j in range (50):
      if i==j:
        row.append(0)
      else :
        row.append(random.randint(1,5))
    time_matrix.append(row)

def print_travel_chromosomes():
  global travel_chromosomes
  for i in travel_chromosomes:
    print(i)



def create_travel_chromosomes():
    global travel_chromosomes
    coint = 0
    for i in range(5):
        chromosome = []
        key_list = list(visited_attractions.keys())

        random_length = 10
        while len(chromosome) < random_length:
            random_attraction = random.choice(key_list)
            if visited_attractions[random_attraction][3] == 0:
                chromosome.append(random_attraction)
                visited_attractions[random_attraction][3] = 1
        travel_chromosomes.append(chromosome)

def travel_distance(chromosome):
  global time_matrix
  global visited_attractions
  total_distance = 0
  for i in range(len(chromosome)-1):
    current_index = get_key_index(chromosome[i])
    next_index = get_key_index(chromosome[i+1])
    total_distance += time_matrix[current_index][next_index]
  return total_distance


def fitness_check(chromosome):
  global visited_attractions
  global cost_factor
  global popularity_factor
  number_of_sites_visited = len(chromosome)
  total_hours= 0
  total_populairty = 0
  total_cost = 0
  penalty = 0
  repeated_sites = 0
  for site in chromosome:
    total_hours += visited_attractions[site][2]
    total_populairty += visited_attractions[site][1]
    total_cost += visited_attractions[site][0]
    if visited_attractions[site][3] > 1:
      repeated_sites += 1


  total_hours += travel_distance(chromosome)
  if total_hours >8:
    penalty = (total_hours - 8) *0.5
  if repeated_sites > 0:
    penalty += repeated_sites *0.25
  fitness = total_populairty*popularity_factor + total_cost*cost_factor - penalty
  return fitness



def cal_fitness_all_pop():
  global travel_chromosomes
  global travel_day_fitness
  for i in travel_chromosomes:
    travel_day_fitness.append(fitness_check(i))


def selection():
  global travel_chromosomes
  global travel_day_fitness
  mother = []
  father= []
  temp  = travel_day_fitness.copy()
  best_index_1 = temp.index(max(temp))
  mother = travel_chromosomes[best_index_1]
  temp.remove(max(temp))
  best_index_2 = temp.index(max(temp))
  father = travel_chromosomes[best_index_2]

  return mother ,father



def travel_crossover(mother, father):
     offspring1 = []
     offspring2 = []
     crossover_point = random.randint(1, len(mother) - 1)
     offspring1 = mother[:crossover_point] + father[crossover_point:]
     offspring2 = father[:crossover_point] + mother[crossover_point:]
     fit1 = fitness_check(offspring1)
     fit2 = fitness_check(offspring2)
     fittest_fit = None
     fittest_offsprint = None
     worst = min(travel_day_fitness)
     if fit1 > fit2:
        fittest = fit1
        fittest_offsprint = offspring1
     else :
      fittest = fit2
      fittest_offsprint = offspring2
     return fittest_offsprint



def travel_mutation(chromosome):
    mask_value = 0.25
    global visited_attractions
    key_list = list(visited_attractions.keys())

    for i in range(len(chromosome)):
        prob = random.random()
        if prob < mask_value:
            random_attraction = random.choice(key_list)
            while random_attraction in chromosome:
                random_attraction = random.choice(key_list)

            visited_attractions[random_attraction][3] += 1
            chromosome[i] = random_attraction

    worst = min(travel_day_fitness)
    fittest = fitness_check(chromosome)
    if fittest > worst:
        travel_chromosomes[travel_day_fitness.index(worst)] = chromosome
        travel_day_fitness[travel_day_fitness.index(worst)] = fittest



def recalculate_visits():
  global visited_attractions
  global travel_chromosomes
  for sites in visited_attractions:
    visited_attractions[sites][3] = 0

  for chromosome in travel_chromosomes:
    for site in chromosome:
      visited_attractions[site][3] +=1



def give_happiness_cost (chromosome):
  popularity = 0
  cost = 0
  for site in chromosome:
    popularity += visited_attractions[site][1]
    cost += visited_attractions[site][0]
  return popularity , cost

def output(total_generations,history):
  global travel_chromosomes
  best_chromosome = travel_chromosomes[travel_day_fitness.index(max(travel_day_fitness))]
  best_fitness = max(travel_day_fitness)
  popularity  ,cost = give_happiness_cost(best_chromosome)


  print(f"Total generations:{total_generations}")
  print(f"Best chromosome: {best_chromosome}")
  print(f"Best fitness: {best_fitness}")
  print(f"Popularity: {popularity}")
  print(f"Cost: {cost}$")
  print("------------------------------------------------------")
  print("Remaining 4 days summary:")
  for i in range (len(travel_chromosomes)):
    if i != travel_day_fitness.index(max(travel_day_fitness)):
      popularity  ,cost = give_happiness_cost(travel_chromosomes[i])
      print(f"Chromosome: {travel_chromosomes[i]}")
      print(f"Fitness: {travel_day_fitness[i]}")
      print(f"Popularity: {popularity}")
      print(f"Cost: {cost}$")


  print("------------------------------------------------------")
  print("best solutions and fitness generation wise:")
  for i in range (len(history)):
    print(f"Generation: {i+1}")
    print(f"Best chromosome: {history[i][1]}")
    print(f"Best fitness: {history[i][0]}")
    print("------------------------------------------------------")



def evolution():
  criterai= 0
  fitness_and_sol = []
  max_fitness = max(travel_day_fitness)
  number_of_generations = 0
  while criterai < 50:

    mother , father = selection()
    chromosome = travel_crossover(mother, father)
    travel_mutation(chromosome)
    recalculate_visits()
    if max_fitness == max(travel_day_fitness):
      criterai +=1
    else :
      criterai = 0
    max_fitness = max(travel_day_fitness)
    number_of_generations +=1

    top_fitness = max(travel_day_fitness)
    top_index = travel_day_fitness.index(top_fitness)
    top_chromosome = travel_chromosomes[top_index]
    fitness_and_sol.append((top_fitness, top_chromosome))
  output(number_of_generations,fitness_and_sol)


create_matrix()
create_travel_chromosomes()
cal_fitness_all_pop()
evolution()



Total generations:123
Best chromosome: ['Chaukhandi Tombs', 'Harappa Archaeological Site', 'Swat Valley', 'Makli Necropolis', 'Tomb of Allama Iqbal', 'Mazar-e-Quaid', 'Lahore Fort', 'Minar-e-Pakistan', 'Ziarat Juniper Forest', 'K2 Base Camp']
Best fitness: 21.25
Popularity: 85
Cost: 72$
------------------------------------------------------
Remaining 4 days summary:
Chromosome: ['Chaukhandi Tombs', 'Harappa Archaeological Site', 'Swat Valley', 'Siri Paye', 'Shalimar Gardens', 'Mazar-e-Quaid', 'Lahore Fort', 'Minar-e-Pakistan', 'Badshahi Mosque', 'K2 Base Camp']
Fitness: 21.0
Popularity: 88
Cost: 75$
Chromosome: ['Chaukhandi Tombs', 'Harappa Archaeological Site', 'Chitral Fort', 'Makli Necropolis', 'Tomb of Allama Iqbal', 'Mazar-e-Quaid', 'Lahore Fort', 'Minar-e-Pakistan', 'Ziarat Juniper Forest', 'K2 Base Camp']
Fitness: 21.25
Popularity: 84
Cost: 76$
Chromosome: ['Chaukhandi Tombs', 'Harappa Archaeological Site', 'Swat Valley', 'Buddhist Monastery', 'Tomb of Allama Iqbal', 'Mazar-e-Qu